# 

M
o
n
i
t
o
r
e
o

d
e

C
a
m
b
i
o
s

A
g
r
í
c
o
l
a
s

c
o
n

S
e
n
t
i
n
e
l
-
1

S
A
R
# 
# 

A
n
á
l
i
s
i
s

C
o
m
p
l
e
t
o

-

L
l
a
n
o
s

O
r
i
e
n
t
a
l
e
s

d
e

C
o
l
o
m
b
i
a
# 
# 
# 

R
e
s
u
m
e
n
N
o
t
e
b
o
o
k

c
o
n
s
o
l
i
d
a
d
o

q
u
e

i
n
t
e
g
r
a

p
r
e
p
a
r
a
c
i
ó
n

d
e

d
a
t
o
s
,

p
r
e
p
r
o
c
e
s
a
m
i
e
n
t
o

S
e
n
t
i
n
e
l
-
1
,

d
e
t
e
c
c
i
ó
n

d
e

c
a
m
b
i
o
s

y

v
i
s
u
a
l
i
z
a
c
i
ó
n
.
*
*
M
u
n
i
c
i
p
i
o
s
:
*
*

9

e
n

C
a
s
a
n
a
r
e

y

M
e
t
a

*
*
M
e
t
o
d
o
l
o
g
í
a
:
*
*

C
a
n
t
y

e
t

a
l
.

(
2
0
2
0
)
,

C
o
n
r
a
d
s
e
n

e
t

a
l
.

(
2
0
0
3
)

*
*
P
l
a
t
a
f
o
r
m
a
:
*
*

G
o
o
g
l
e

E
a
r
t
h

E
n
g
i
n
e



# 
# 

1
.

C
o
n
f
i
g
u
r
a
c
i
ó
n

d
e
l

E
n
t
o
r
n
o
# 
# 
# 

I
n
s
t
a
l
a
c
i
ó
n

d
e

D
e
p
e
n
d
e
n
c
i
a
s
`
`
`
b
a
s
h
p
i
p

i
n
s
t
a
l
l

e
a
r
t
h
e
n
g
i
n
e
-
a
p
i

g
e
e
m
a
p

g
e
o
p
a
n
d
a
s

f
i
o
n
a

m
a
t
p
l
o
t
l
i
b

s
e
a
b
o
r
n

p
a
n
d
a
s

n
u
m
p
y

p
l
o
t
l
y

j
u
p
y
t
e
r
`
`
`



In [ ]:
# Importar librerías
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook', font_scale=1.1)
sns.set_palette('Set2')

Path('data').mkdir(exist_ok=True)
print('✓ Librerías cargadas')

In [ ]:
# Inicializar Google Earth Engine
try:
    ee.Initialize()
    print('✓ Earth Engine inicializado')
except:
    print('⚠ Ejecute: ee.Authenticate()')
    raise

## 2. Preparación de Datos y Área de Estudio

### 2.1. Configuración de Parámetros

In [ ]:
# Parámetros temporales - Segundo Semestre 2025
FECHA_INICIO = '2025-01-01'
FECHA_FIN = '2025-09-30'

# Períodos para análisis de cambios
REFERENCE_START = '2025-01-01'  # Primer semestre 2025 (referencia)
REFERENCE_END = '2025-06-30'
TARGET_START = '2025-07-01'     # Segundo semestre 2025 (análisis)
TARGET_END = '2025-09-30'

# FeatureCollection de Earth Engine con municipios de Colombia
EE_ASSET = 'users/geoia/MapasBase/ColMuni'

# Códigos de municipios objetivo
CODIGOS_MUNICIPIOS = [
    '50573',  # Puerto López
    '50150',  # Castilla La Nueva
    '50680',  # San Carlos de Guaroa
    '50124',  # Cabuyaro
    '85410',  # Tauramena
    '85001',  # Yopal
    '85010',  # Aguazul
    '85225',  # Nunchía
    '85440'   # Villanueva
]

print(f'Período total: {FECHA_INICIO} a {FECHA_FIN}')
print(f'Referencia (1er semestre): {REFERENCE_START} a {REFERENCE_END}')
print(f'Análisis (2do semestre): {TARGET_START} a {TARGET_END}')
print(f'Municipios a analizar: {len(CODIGOS_MUNICIPIOS)}')

## 2.2. Carga de Municipios desde Earth Engine

In [ ]:
# Cargar FeatureCollection de Earth Engine
municipios_col = ee.FeatureCollection(EE_ASSET)

# Filtrar municipios objetivo
municipios_filtrados = municipios_col.filter(
    ee.Filter.inList('MPIO_CCDGO', CODIGOS_MUNICIPIOS)
)

# Obtener información de municipios
n_municipios = municipios_filtrados.size().getInfo()
print(f'✓ Municipios cargados desde Earth Engine: {n_municipios}')

# Mostrar lista de municipios
municipios_info = municipios_filtrados.select(['DPTO_CNMBR', 'MPIO_CNMBR', 'MPIO_CCDGO']).getInfo()
print('\nMunicipios seleccionados:')
for feat in municipios_info['features']:
    props = feat['properties']
    print(f"  {props['MPIO_CNMBR']:<25} ({props['DPTO_CNMBR']:<10}) - Código: {props['MPIO_CCDGO']}")

## 2.3. Crear AOI para Earth Engine

In [ ]:
# Crear geometría única (unión de todos los municipios)
aoi = municipios_filtrados.geometry()

# Calcular centroide y bounds
bounds = aoi.bounds().getInfo()
centroid_coords = aoi.centroid().coordinates().getInfo()

print(f'✓ AOI creada')
print(f'Centroide: {centroid_coords[1]:.4f}, {centroid_coords[0]:.4f}')
print(f'Bounds: {bounds}')

# 
# 

3
.

P
r
e
p
r
o
c
e
s
a
m
i
e
n
t
o

S
e
n
t
i
n
e
l
-
1
# 
# 
# 

3
.
1
.

D
e
f
i
n
i
r

P
i
p
e
l
i
n
e

d
e

P
r
o
c
e
s
a
m
i
e
n
t
o



In [ ]:
def process_sentinel1(aoi, start, end, orbit=None):
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi).filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    
    if orbit:
        s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit))
    
    def to_dB(img):
        vv = ee.Image(10).multiply(img.select('VV').log10()).rename('VV')
        vh = ee.Image(10).multiply(img.select('VH').log10()).rename('VH')
        ratio = vv.subtract(vh).rename('VV_VH_ratio')
        return img.addBands([vv, vh, ratio]).copyProperties(img, ['system:time_start'])
    
    def speckle_filter(img):
        k = ee.Kernel.square(3, 'pixels')
        vv = img.select('VV').focal_median(k).rename('VV_filt')
        vh = img.select('VH').focal_median(k).rename('VH_filt')
        ratio = img.select('VV_VH_ratio').focal_median(k).rename('ratio_filt')
        return img.addBands([vv, vh, ratio])
    
    return s1.map(to_dB).map(speckle_filter)

print('✓ Pipeline definido')

# 
# 
# 

3
.
2
.

C
a
r
g
a
r

y

F
i
l
t
r
a
r

C
o
l
e
c
c
i
ó
n



In [ ]:
# Cargar colección
s1_col = process_sentinel1(aoi, FECHA_INICIO, FECHA_FIN)

# Seleccionar órbita con más imágenes
s1_asc = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
s1_desc = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

n_asc = s1_asc.size().getInfo()
n_desc = s1_desc.size().getInfo()

if n_asc >= n_desc:
    s1_filtered = s1_asc
    orbit_type = 'ASCENDING'
else:
    s1_filtered = s1_desc
    orbit_type = 'DESCENDING'

print(f'Órbita seleccionada: {orbit_type}')
print(f'Imágenes disponibles: {s1_filtered.size().getInfo()}')

# 
# 

4
.

D
e
t
e
c
c
i
ó
n

d
e

C
a
m
b
i
o
s
# 
# 
# 

4
.
1
.

C
r
e
a
r

C
o
m
p
o
s
i
c
i
o
n
e
s



In [ ]:
# Composiciones de referencia y análisis
# IMPORTANTE: No usar .clip() aquí - las estadísticas se extraerán por municipio individual
ref_comp = s1_filtered.filterDate(REFERENCE_START, REFERENCE_END).median()
target_comp = s1_filtered.filterDate(TARGET_START, TARGET_END).median()

n_ref = s1_filtered.filterDate(REFERENCE_START, REFERENCE_END).size().getInfo()
n_target = s1_filtered.filterDate(TARGET_START, TARGET_END).size().getInfo()

print(f'Composición referencia: {n_ref} imágenes')
print(f'Composición análisis: {n_target} imágenes')
print('\nNota: Las estadísticas se extraerán por municipio individual (sin áreas intermedias)')

# 
# 
# 

4
.
2
.

M
é
t
o
d
o
s

d
e

D
e
t
e
c
c
i
ó
n
# 
# 
# 
# 

M
é
t
o
d
o

1
:

D
i
f
e
r
e
n
c
i
a
s

T
e
m
p
o
r
a
l
e
s



In [ ]:
# Diferencias absolutas
diff_vv = target_comp.select('VV_filt').subtract(ref_comp.select('VV_filt')).rename('diff_VV')
diff_vh = target_comp.select('VH_filt').subtract(ref_comp.select('VH_filt')).rename('diff_VH')

# Magnitud de cambio
change_mag = diff_vv.pow(2).add(diff_vh.pow(2)).sqrt().rename('change_magnitude')

print('✓ Diferencias calculadas')

# 
# 
# 
# 

M
é
t
o
d
o

2
:

Í
n
d
i
c
e

N
D
C
V



In [ ]:
def calc_ndcv(band):
    ref_lin = ee.Image(10).pow(ref_comp.select(band).divide(10))
    target_lin = ee.Image(10).pow(target_comp.select(band).divide(10))
    return target_lin.subtract(ref_lin).abs().divide(target_lin.add(ref_lin))

ndcv_vv = calc_ndcv('VV_filt').rename('NDCV_VV')
ndcv_vh = calc_ndcv('VH_filt').rename('NDCV_VH')
ndcv_combined = ndcv_vv.add(ndcv_vh).divide(2).rename('NDCV')

# Máscara de cambio
change_mask = ndcv_combined.gt(0.3).rename('change_mask')

print('✓ NDCV calculado')

# 
# 
# 
# 

M
é
t
o
d
o

3
:

C
l
a
s
i
f
i
c
a
c
i
ó
n

d
e

C
a
m
b
i
o
s



In [ ]:
# Umbrales
STRONG_THRESH = 3.0
MOD_THRESH = 1.5

# Clasificación (sin recortar - se procesará por municipio individual)
change_class = ee.Image(0)
change_class = change_class.where(diff_vv.gt(STRONG_THRESH), 1)  # Aumento fuerte
change_class = change_class.where(diff_vv.lt(-STRONG_THRESH), 2)  # Disminución fuerte
change_class = change_class.where(
    diff_vv.gt(MOD_THRESH).And(diff_vv.lte(STRONG_THRESH)), 3
)  # Aumento moderado
change_class = change_class.where(
    diff_vv.lt(-MOD_THRESH).And(diff_vv.gte(-STRONG_THRESH)), 4
)  # Disminución moderada
change_class = change_class.rename('change_class')

print('✓ Clasificación completada')
print('Nota: Las capas se procesarán dentro de cada polígono municipal')

# 
# 

5
.

E
s
t
a
d
í
s
t
i
c
a
s

p
o
r

M
u
n
i
c
i
p
i
o



In [ ]:
# Extraer estadísticas por municipio
stats_list = []

# Obtener features como lista
features_list = municipios_filtrados.toList(municipios_filtrados.size())

for i in range(n_municipios):
    feature = ee.Feature(features_list.get(i))
    geom = feature.geometry()
    props = feature.toDictionary().getInfo()
    
    # Estadísticas de diferencia
    diff_stats = diff_vv.addBands(diff_vh).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    # NDCV
    ndcv_mean = ndcv_combined.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=10,
        maxPixels=1e9
    ).getInfo().get('NDCV')
    
    # Área de cambio
    area_total = geom.area().divide(10000).getInfo()  # ha
    area_cambio = change_mask.multiply(ee.Image.pixelArea()).divide(10000).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=10,
        maxPixels=1e9
    ).getInfo().get('change_mask')
    
    if area_cambio is None:
        area_cambio = 0
    
    stats = {
        'municipio': props['MPIO_CNMBR'],
        'departamento': props['DPTO_CNMBR'],
        'codigo': props['MPIO_CCDGO'],
        'diff_VV': diff_stats.get('diff_VV'),
        'diff_VH': diff_stats.get('diff_VH'),
        'NDCV': ndcv_mean,
        'area_total_ha': area_total,
        'area_cambio_ha': area_cambio,
        'pct_cambio': (area_cambio / area_total * 100) if area_total > 0 else 0
    }
    
    stats_list.append(stats)
    print(f'  Procesado: {props["MPIO_CNMBR"]} ({props["DPTO_CNMBR"]})')

stats_df = pd.DataFrame(stats_list).sort_values('pct_cambio', ascending=False)
print(f'\n✓ Estadísticas calculadas para {len(stats_df)} municipios')
print('\n' + stats_df.to_string(index=False))

# 
# 

6
.

V
i
s
u
a
l
i
z
a
c
i
ó
n
# 
# 
# 

6
.
1
.

M
a
p
a

I
n
t
e
r
a
c
t
i
v
o



In [ ]:
Map = geemap.Map(center=[centroid_coords[1], centroid_coords[0]], zoom=9)

# Crear máscara de municipios (solo áreas dentro de polígonos municipales)
municipios_mask = ee.Image.constant(1).clip(municipios_filtrados.geometry())

# Aplicar máscara a las capas de análisis para mostrar SOLO dentro de municipios
diff_vv_masked = diff_vv.updateMask(municipios_mask)
ndcv_masked = ndcv_combined.updateMask(municipios_mask)
change_class_masked = change_class.updateMask(municipios_mask)

# Agregar municipios con borde visible
Map.addLayer(municipios_filtrados, {'color': 'black'}, 'Límites Municipales', True, 0.8)

# Parámetros de visualización
vis_diff = {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}
vis_ndcv = {'min': 0, 'max': 0.6, 'palette': ['white', 'yellow', 'orange', 'red']}
vis_class = {'min': 0, 'max': 4, 'palette': ['gray', 'blue', 'red', 'lightblue', 'orange']}

# Agregar capas ENMASCARADAS (solo dentro de municipios)
Map.addLayer(diff_vv_masked, vis_diff, 'Diferencia VV (solo municipios)', True)
Map.addLayer(ndcv_masked, vis_ndcv, 'NDCV (solo municipios)', True)
Map.addLayer(change_class_masked, vis_class, 'Clasificación (solo municipios)', False)

# Leyenda
legend_dict = {
    'Sin cambio': 'gray',
    'Aumento fuerte (>3dB)': 'blue',
    'Disminución fuerte (<-3dB)': 'red',
    'Aumento moderado': 'lightblue',
    'Disminución moderada': 'orange'
}
Map.add_legend(legend_dict=legend_dict, title='Clasificación de Cambios')

Map

# 
# 
# 

6
.
2
.

G
r
á
f
i
c
o
s

E
s
t
a
d
í
s
t
i
c
o
s



In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Porcentaje de cambio
colors = ['#1f77b4' if d == 'CASANARE' else '#ff7f0e' for d in stats_df['departamento']]
axes[0].barh(range(len(stats_df)), stats_df['pct_cambio'], color=colors)
axes[0].set_yticks(range(len(stats_df)))
axes[0].set_yticklabels(stats_df['municipio'])
axes[0].set_xlabel('Porcentaje de Área con Cambio (%)')
axes[0].set_title('Detección de Cambios por Municipio', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Área en hectáreas
axes[1].barh(range(len(stats_df)), stats_df['area_cambio_ha'], color=colors)
axes[1].set_yticks(range(len(stats_df)))
axes[1].set_yticklabels(stats_df['municipio'])
axes[1].set_xlabel('Área con Cambio (hectáreas)')
axes[1].set_title('Área Total con Cambios Detectados', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('data/cambios_municipios.png', dpi=300, bbox_inches='tight')
plt.show()
print('✓ Gráficos guardados')

# 
# 

7
.

E
x
p
o
r
t
a
c
i
ó
n

d
e

R
e
s
u
l
t
a
d
o
s



In [ ]:
# Guardar estadísticas
stats_df.to_csv('data/estadisticas_cambios.csv', index=False)

# Guardar parámetros
params = {
    'fecha_inicio': FECHA_INICIO,
    'fecha_fin': FECHA_FIN,
    'reference_period': {'start': REFERENCE_START, 'end': REFERENCE_END},
    'target_period': {'start': TARGET_START, 'end': TARGET_END},
    'ee_asset': EE_ASSET,
    'codigos_municipios': CODIGOS_MUNICIPIOS,
    'orbit_type': orbit_type,
    'n_images_ref': n_ref,
    'n_images_target': n_target,
    'strong_threshold': STRONG_THRESH,
    'moderate_threshold': MOD_THRESH,
    'centroid': centroid_coords,
    'bounds': bounds
}

with open('data/parametros_analisis.json', 'w') as f:
    json.dump(params, f, indent=2)

print('✓ Resultados exportados en data/')
print(f'  - estadisticas_cambios.csv')
print(f'  - parametros_analisis.json')
print(f'  - cambios_municipios.png')

### Referencias bibliográficas
# 
# 

R
e
s
u
m
e
n

y

C
o
n
c
l
u
s
i
o
n
e
s
# 
# 
# 

P
r
o
d
u
c
t
o
s

G
e
n
e
r
a
d
o
s
1
.

✓

E
s
t
a
d
í
s
t
i
c
a
s

d
e

c
a
m
b
i
o

p
o
r

m
u
n
i
c
i
p
i
o

(
C
S
V
)
2
.

✓

P
a
r
á
m
e
t
r
o
s

d
e
l

a
n
á
l
i
s
i
s

(
J
S
O
N
)
3
.

✓

C
a
p
a

e
s
p
a
c
i
a
l

c
o
n

r
e
s
u
l
t
a
d
o
s

(
G
e
o
P
a
c
k
a
g
e
)
4
.

✓

V
i
s
u
a
l
i
z
a
c
i
o
n
e
s

(
P
N
G
)
5
.

✓

M
a
p
a

i
n
t
e
r
a
c
t
i
v
o

(
g
e
e
m
a
p
)
# 
# 
# 

I
n
t
e
r
p
r
e
t
a
c
i
ó
n

A
g
r
í
c
o
l
a
*
*
C
a
m
b
i
o
s

d
e
t
e
c
t
a
d
o
s
:
*
*
-

*
*
A
u
m
e
n
t
o

d
e

b
a
c
k
s
c
a
t
t
e
r

V
V
/
V
H
*
*
:

P
o
s
i
b
l
e

c
r
e
c
i
m
i
e
n
t
o

v
e
g
e
t
a
t
i
v
o
,

e
m
e
r
g
e
n
c
i
a

d
e

c
u
l
t
i
v
o
s
,

i
n
u
n
d
a
c
i
ó
n

d
e

c
a
m
p
o
s

(
p
r
e
p
a
r
a
c
i
ó
n

a
r
r
o
z
)
-

*
*
D
i
s
m
i
n
u
c
i
ó
n

d
e

b
a
c
k
s
c
a
t
t
e
r
*
*
:

P
o
s
i
b
l
e

c
o
s
e
c
h
a
,

s
e
n
e
s
c
e
n
c
i
a
,

p
r
e
p
a
r
a
c
i
ó
n

d
e

s
u
e
l
o
,

s
e
q
u
í
a
-

*
*
A
l
t
a

v
a
r
i
a
b
i
l
i
d
a
d
*
*
:

R
o
t
a
c
i
ó
n

d
e

c
u
l
t
i
v
o
s
,

g
e
s
t
i
ó
n

a
g
r
í
c
o
l
a

a
c
t
i
v
a
# 
# 
# 

R
e
f
e
r
e
n
c
i
a
s

C
o
m
p
l
e
t
a
s
V
e
r

a
r
c
h
i
v
o

`
r
e
f
e
r
e
n
c
e
s
.
b
i
b
`

p
a
r
a

b
i
b
l
i
o
g
r
a
f
í
a

e
n

f
o
r
m
a
t
o

B
i
b
T
e
X
.
*
*
R
e
f
e
r
e
n
c
i
a
s

c
l
a
v
e
:
*
*
-

C
a
n
t
y

e
t

a
l
.

(
2
0
2
0
)
:

h
t
t
p
s
:
/
/
d
o
i
.
o
r
g
/
1
0
.
3
3
9
0
/
r
s
1
2
0
1
0
0
4
6
-

C
o
n
r
a
d
s
e
n

e
t

a
l
.

(
2
0
0
3
)
:

h
t
t
p
s
:
/
/
d
o
i
.
o
r
g
/
1
0
.
1
1
0
9
/
T
G
R
S
.
2
0
0
2
.
8
0
8
0
6
6
-

V
e
l
o
s
o

e
t

a
l
.

(
2
0
1
7
)
:

h
t
t
p
s
:
/
/
d
o
i
.
o
r
g
/
1
0
.
1
0
1
6
/
j
.
r
s
e
.
2
0
1
7
.
0
7
.
0
1
5
